In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd

Берем картинки для тестирования из произвольного датасета

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

test_images = test_images / 255.0

В ходе анализа CNN сетей для параметров регрессора было решено взять следующие:
    1. Количество фильтров в каждом слое
    2. Размер ядра свертки
    3. Количество слоев
    4. Количество выходных классов
    5. Padding
    

In [ ]:
import time 
n=2
m=2
l=4  # количество слоев
classes=10
padding=["valid","same"]
data={}
df = pd.DataFrame(data)
for k in range(l):
    for i in range(n):
        for j in range(m):
            for ind_p, p in enumerate(padding):
                for c in range(2,classes,1):
                    model = models.Sequential()
                    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
                    filters=(16*2**i)  #количество фильтров
                    kernel_square=(j+1)**2 # площадь ядра свертки
                    n_layers=k+1   # количество слоев
                    for _ in range(k):
                        model.add(layers.Conv2D(4*2**i, j+1, j+1, activation='relu'))
                        model.add(layers.MaxPooling2D((2, 2)))
                    
                
                    model.add(layers.Flatten())
                    model.add(layers.Dense(64, activation='relu'))
                    model.add(layers.Dense(c))
                    
                    average_time=0
                
                    for _ in range(10):
                        start = time.time() ## точка отсчета времени
                        test_loss = model.predict(test_images[:5])
                        end = time.time() - start ## собственно время работы программы
                        average_time+=end
                    average_time/=10



                    
            
                    new_row = {'filters': filters, 'kernal_square': kernel_square,'n_layers': n_layers, 'padding': ind_p, 'out_classes': c, 'time': average_time}
            
                    df = df.append(new_row, ignore_index=True)
        

        

Для первого слоя устанавливаем input shape в соответствии с картинками, которые мы подаем на вход. На выходе получаем датасет для регрессора:

In [ ]:
df

Создание регрессора:

Для начала необходимо взять датасет из предыдущей задачи и разбить данные на тренировочные и валидационные:

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('time', axis=1)
y = df['time']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

В качестве оченки качества регрессии было решено взять Среднюю квадратичную ошибку, чтобы избежать грубых ошибок в предсказании времени, так как при генерации моделей могли произойти не маленькие отклонения.

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

Добавление оставшихся необходимых импортов:

In [ ]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

Обучение регрессора. В данном скрипте происходит компиляция модели и обучение на 100 эпохах

In [ ]:
tf.random.set_seed(42)

model = Sequential([
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1)
])

model.compile(
    loss=rmse,
    optimizer=Adam(),
    metrics=[rmse]
)

model.fit(X_train, y_train, epochs=100)

In [ ]:
import numpy as np
predictions = model.predict(X_test)
predictions = np.ravel(predictions)
print(X_test[:10])
predictions[:10]

Как мы видим в процессе обучения регрессор уменьшает RMSE, а так же не доходит до стадии переобучения. При валидации тестовой выборки не было выявлено существенных отклонений предсказателя. Для получения более качественных результатов стоит уделить больше внимания созданию датасета и первой части задания, а так же "поиграться" c выбором оптимизатора, количества слоев, функциями активации и т.п